In [6]:
from typing import Dict, List
from io import StringIO
import csv
import requests
import os
import itertools
import tiktoken
import openai
import pandas as pd

# from utilities import num_token_from_messages, memoize_sqlite

from helpers.notebook.embeddings import (
    get_embedding,
    distances_from_embeddings,
    indices_of_nearest_neighbors_from_distances
)